# Operationnal Pipeline

In [ ]:
from fds.config import set_url, set_api_key
set_url("https://api.spacetower.exotrail.space/fds/v1")
set_api_key('MY_API_KEY')

In [ ]:
from datetime import datetime, UTC

import pandas as pd

from fds.models.determination.configuration import OrbitDeterminationConfiguration
from fds.models.determination.use_case import OrbitDetermination
from fds.models.ground_station import GroundStation
from fds.models.orbit_extrapolation.requests import EventsRequestStationVisibility, OemRequest
from fds.models.orbit_extrapolation.use_case import OrbitExtrapolation
from fds.models.orbital_state import PropagationContext, CovarianceMatrix, OrbitalState
from fds.models.orbits import OrbitType
from fds.models.spacecraft import Battery, SolarArray, ThrusterElectrical, SpacecraftBox
from fds.models.telemetry import TelemetryGpsNmea
from fds.models.two_line_element import TwoLineElement
from fds.utils.frames import Frame

## Creation of FDS objects

In [ ]:
battery = Battery(
    depth_of_discharge=0.3,  # 0<x<1
    nominal_capacity=560,  # W
    minimum_charge_for_firing=0.9,  # 0<x<1
    initial_charge=1,  # 0<x<1
)

solar_array = SolarArray(
    kind="DEPLOYABLE_FIXED",
    initialisation_kind=SolarArray.InitialisationKind.MAXIMUM_POWER,
    efficiency=.293,  # 0<x<1
    normal_in_satellite_frame=(0, 0, -1),  # Unit vector
    maximum_power=300,  # W
    surface=0.7490056949399673,  # m^2
)

electrical_thruster = ThrusterElectrical(
    isp=950,  # s
    thrust=0.005,  # N
    axis_in_satellite_frame=(-1, 0, 0),  # Unit vector
    propellant_mass=4,  # kg
    wet_mass=11,  # kg
    warm_up_duration=240,  # s
    maximum_thrust_duration=1200,  # s
    impulse=37265.27,  # Ns
    power=300,  # W
    stand_by_power=1.1,  # W
    warm_up_power=50,  # W
)

spacecraft = SpacecraftBox(
    battery=battery,
    thruster=electrical_thruster,
    solar_array=solar_array,
    platform_mass=112,  # kg
    drag_coefficient=2.2,
    length_x=.5,  # m
    length_y=.5,  # m
    length_z=.5,  # m
    max_angular_velocity=2,  # deg/s
    max_angular_acceleration=.5,  # deg/s^2
)

prop_context = PropagationContext(
    model_perturbations=[
        PropagationContext.Perturbation.DRAG,
        PropagationContext.Perturbation.EARTH_POTENTIAL,
        PropagationContext.Perturbation.SRP,
        PropagationContext.Perturbation.THIRD_BODY,
    ],
    model_solar_flux=150,  # SFU
    model_earth_potential_deg=30,
    model_earth_potential_ord=30,
    model_atmosphere_kind=PropagationContext.AtmosphereModel.HARRIS_PRIESTER,
    integrator_kind=PropagationContext.IntegratorKind.DORMAND_PRINCE_853,
    integrator_min_step=0.01,  # s
    integrator_max_step=100,  # s
)

## Telemetry processing


In [ ]:

columns = ["Time", "gpsUtcTime", "gpsUtcTimeFraction", "gpsLatitude", "gpsLongitude", "gpsAltitudeMeters",
           "gpsGroundSpeed", "gpsHDilutionOfPos", "gpsGeoid"]
data = [[1699733402000, 1699732645, 0, 76.65963745117188, 2.1627416610717773, 541560.9375, 27579.505859375, 2,
         17.399999618530273],
        [1699734002000, 1699734002, 0, 62.512508392333984, -132.4083709716797, 537359.0625, 27597.474609375,
         0.6000000238418579, 3.9000000953674316],
        [1699734602000, 1699734602, 0, 25.359493255615234, -145.8915557861328, 525933.75, 27650.193359375,
         0.6000000238418579, -16.399999618530273],
        [1699735202000, 1699735202, 0, -12.387834548950195, -153.5818634033203, 523971.09375, 27659.669921875, 0.5,
         4.400000095367432],
        [1699735802000, 1699735802, 0, -49.87264633178711, -163.34860229492188, 535089.9375, 27608.986328125,
         0.800000011920929, -18],
        [1699736402000, 1699736402, 0, -82.33123016357422, 127.23735809326172, 543933, 27569.248046875, 0.5,
         -38.70000076293945],
        [1699737003000, 1699737002, 0, -53.56744384765625, 30.74840545654297, 538869.0625, 27593.3125,
         0.6000000238418579, 36.400001525878906],
        [1699737603000, 1699737602, 0, -16.198204040527344, 20.234773635864258, 528751, 27640.736328125, 0.5,
         13.800000190734863],
        [1699738203000, 1699738202, 0, 21.511497497558594, 12.611784934997559, 528960.375, 27639.123046875,
         0.6000000238418579, 23.299999237060547],
        [1699738803000, 1699738803, 0, 58.82115173339844, 0.5973266959190369, 538318.125, 27593.30859375,
         3.4000000953674316, 46.900001525878906]]

datas = pd.DataFrame(data, columns=columns)
datas["gpsGroundSpeed"] = datas["gpsGroundSpeed"].apply(lambda x: x * 5 / 18)

datas_filtered = datas[["gpsLatitude", "gpsLongitude", "gpsGroundSpeed", "gpsAltitudeMeters", "gpsGeoid"]]

nmea_measurements = [list(d) for d in list(datas_filtered.values)]
dates = [datetime.fromtimestamp(ts, tz=UTC).strftime("%Y-%m-%dT%H:%M:%S.%fZ") for ts in datas["gpsUtcTime"].to_list()]

initial_tle = TwoLineElement(
    line_1="1 99782U          23315.82599410 -.00000001  00000-0  00000-0 0 00002",
    line_2="2 99782 097.4747 028.7679 0008388 247.8859 112.9940 15.14750288000012",
)

telemetry_nmea = TelemetryGpsNmea(
    measurements=nmea_measurements,
    dates=dates,
    standard_deviation_altitude=200,
    standard_deviation_ground_speed=5,
    standard_deviation_latitude=0.002,
    standard_deviation_longitude=0.002
)

## Run Orbit Determination

In [ ]:
process_noise_matrix = CovarianceMatrix.from_diagonal(
    diagonal=(1E-1, 1E-1, 1E-1, 1E-4, 1E-4, 1E-4),
    frame=Frame.TNW,
    orbit_type=OrbitType.KEPLERIAN,
    date=dates[0]
)

od_config = OrbitDeterminationConfiguration(
    tuning_alpha=0.5,  # Defines the spread of the sigma points. Typical values from 1E-4 to 1E-1.
    tuning_beta=2,  # Beta = 2 is optimal for Gaussian distributions.
    tuning_kappa=-2,  # Defines the spread of the sigma points. Typical values from 1 to 10.
    outliers_manager_scale=10,  #
    outliers_manager_warmup=0,  #
    noise_provider_kind=OrbitDeterminationConfiguration.NoiseProviderKind.BASIC,
    process_noise_matrix=process_noise_matrix,
)

initial_covariance_matrix = CovarianceMatrix.from_diagonal(
    diagonal=(100, 100, 100, 0.1, 0.1, 0.1),
    date=process_noise_matrix.date,
    orbit_type=OrbitType.KEPLERIAN,
    frame=Frame.TNW
)

initial_orbital_state = OrbitalState.from_tle(
    tle=initial_tle,
    covariance_matrix=initial_covariance_matrix,
    propagation_context=prop_context,
    spacecraft=spacecraft
)

orbit_determination = OrbitDetermination(
    initial_orbital_state=initial_orbital_state,
    telemetry=telemetry_nmea,
    configuration=od_config,
)

orbit_determination.run()

In [ ]:
estimated_orbital_state = orbit_determination.result.estimated_orbital_state

ground_station = GroundStation(
    name="Example_station",
    longitude=20,  # deg
    latitude=10,  # deg
    altitude=0.100,  # km
    min_elevation=5,  # deg
)

station_visibility_event = EventsRequestStationVisibility(
    ground_stations=[ground_station],
    start_date=estimated_orbital_state.date,
)

oem_request = OemRequest(
    creator="Exotrail x Promethee",
    object_name="Example",
    object_id="00000",
    frame=Frame.EME2000,
    ephemerides_step=3600,
    write_acceleration=False,
)

## Propagation

In [ ]:
orbit_extrapolation = OrbitExtrapolation(
    initial_orbital_state=estimated_orbital_state,
    station_visibility_events_request=station_visibility_event,
    orbit_data_message_request=oem_request,
    duration=86400,  # s
)

orbit_extrapolation.run()